<a href="https://colab.research.google.com/github/hipeoplea/neiro/blob/master/ekz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

df = pd.read_csv('MNIST_test.csv')
test = df.query('(label == 2) | (label == 3)')
df = pd.read_csv('MNIST_train.csv')
train = df.query('(label == 2) | (label == 3)')
train

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59980,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59983,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59985,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59991,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.decomposition import PCA

pca = PCA(svd_solver='full')
pca.fit(train)
e = pca.explained_variance_ratio_.cumsum()
m = (e>0.83).argmax()+1
m

49

In [11]:

x_train = train.drop(columns=["label"])
y_train = train["label"]

x_test = test.drop(columns=["label"])
y_test = test["label"]

pca = PCA(n_components=m, svd_solver='full')
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

x_train_pca[3, 5]

137.83478555367697

In [24]:
x_train_pca[5, 3]

452.16660489806657

In [13]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
base_learners = [('rf', RandomForestClassifier(random_state=41)), ('svc', SVC(random_state=41))]
final_est = LogisticRegression()

classif = StackingClassifier(estimators=base_learners, final_estimator=final_est)

classif.fit(x_train_pca, y_train)
y_pred = classif.predict(x_test_pca)


In [16]:
from sklearn.metrics import classification_report

classification_report(y_test, y_pred, digits = 3, output_dict=True)["accuracy"]

0.9980411361410382

In [18]:
from sklearn.metrics import confusion_matrix

co_mat = confusion_matrix(y_test, y_pred)
tn = co_mat[0, 0]
fn = co_mat[1, 0]
print(tn)
print(fn)

1030
2


In [20]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
base_learners = [('lr', LogisticRegression(solver='liblinear')),
 ('svc', SVC(random_state=41)),
  ('sgd', SGDClassifier(random_state=41))]
vote = VotingClassifier(estimators=base_learners,
                        voting='hard')
vote.fit(x_train_pca, y_train)
y_pred = vote.predict(x_test_pca)


In [21]:
classification_report(y_test, y_pred, digits = 3, output_dict=True)["accuracy"]

0.9843290891283056

In [23]:
co_mat = confusion_matrix(y_test, y_pred)
tn = co_mat[0, 0]
fn = co_mat[1, 1]
print(tn)
print(fn)

1017
993
